In [1]:
'''
Based on https://www.kaggle.com/justdoit/rossmann-store-sales/xgboost-in-python-with-rmspe/code
Public Score :  0.10826
Private Validation Score :  0.090811
'''

'\nBased on https://www.kaggle.com/justdoit/rossmann-store-sales/xgboost-in-python-with-rmspe/code\nPublic Score :  0.10826\nPrivate Validation Score :  0.090811\n'

In [2]:

import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
import xgboost as xgb
import operator
import matplotlib
matplotlib.use("Agg") #Needed to save figures
import matplotlib.pyplot as plt

In [3]:
def create_feature_map(features):
    outfile = open('xgb.fmap', 'w')
    for i, feat in enumerate(features):
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
    outfile.close()

def rmspe(y, yhat):
    return np.sqrt(np.mean((yhat/y-1) ** 2))

def rmspe_xg(yhat, y):
    y = np.expm1(y.get_label())
    yhat = np.expm1(yhat)
    return "rmspe", rmspe(y,yhat)


In [5]:
def build_features(features, data):
    # remove NaNs
    data.fillna(0, inplace=True)
    data.loc[data.Open.isnull(), 'Open'] = 1
    # Use some properties directly

    # add some more with a bit of preprocessing
    features.append('SchoolHoliday')
    data['SchoolHoliday'] = data['SchoolHoliday'].astype(float)

    features.extend(['StoreType', 'Assortment', 'StateHoliday'])
    mappings = {'0':0, 'a':1, 'b':2, 'c':3, 'd':4}
    data.StoreType.replace(mappings, inplace=True)
    data.Assortment.replace(mappings, inplace=True)
    data.StateHoliday.replace(mappings, inplace=True)

    features.extend(['DayOfWeek', 'month', 'day', 'year'])
    data['year'] = data.Date.dt.year
    data['month'] = data.Date.dt.month
    data['day'] = data.Date.dt.day
    data['DayOfWeek'] = data.Date.dt.dayofweek
    
    features.extend(['competiter'])
    #competiter
    data['competiter'] = ((data['year'] > data['CompetitionOpenSinceYear']) | \
                          ((data['year'] == data['CompetitionOpenSinceYear']) & (data['month'] >=data['CompetitionOpenSinceMonth']))).astype('int')
    
    
    

In [20]:
# calulate statistical value 
def build_sts_features(train, store):
    
    train['values']=train['Sales']/train['Customers']
    store_mean=train.groupby('Store').mean()
    store_mean['Customers_mean'] = store_mean['Customers']
    store_mean['values_mean'] = store_mean['values']
    store_mean['Store']=store_mean.index
    
    store = pd.merge(store, store_mean[['Store','Customers_mean','values_mean']], on='Store')
    return store

In [53]:
print("Load the training, test and store data using pandas")
types = {'CompetitionOpenSinceYear': np.dtype(int),
         'CompetitionOpenSinceMonth': np.dtype(int),
         'StateHoliday': np.dtype(str),
         'Promo2SinceWeek': np.dtype(int),
         'SchoolHoliday': np.dtype(float),
         'PromoInterval': np.dtype(str)}
train = pd.read_csv("data/train.csv", parse_dates=[2], dtype=types)
test = pd.read_csv("data/test.csv", parse_dates=[3], dtype=types)
store = pd.read_csv("data/store.csv")

Load the training, test and store data using pandas


In [54]:
store =build_sts_features(train, store)

In [55]:
print("Assume store open, if not provided")
train.fillna(1, inplace=True)
test.fillna(1, inplace=True)

Assume store open, if not provided


In [56]:
print("Consider only open stores for training. Closed stores wont count into the score.")
train = train[train["Open"] != 0]
print("Use only Sales bigger then zero. Simplifies calculation of rmspe")
train = train[train["Sales"] > 0]

Consider only open stores for training. Closed stores wont count into the score.
Use only Sales bigger then zero. Simplifies calculation of rmspe


In [57]:
print("Join with store")
train = pd.merge(train, store, on='Store')
test = pd.merge(test, store, on='Store')

Join with store


In [58]:
features = []
features.extend(['Store', 'CompetitionDistance', 'CompetitionOpenSinceMonth',
                     'CompetitionOpenSinceYear', 'Promo', 'Promo2', 'Promo2SinceWeek',
                     'Promo2SinceYear','Customers_mean','values_mean'])

print("augment features")
build_features(features, train)
build_features([], test)
print(features)

augment features
['Store', 'CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'Customers_mean', 'values_mean', 'SchoolHoliday', 'StoreType', 'Assortment', 'StateHoliday', 'DayOfWeek', 'month', 'day', 'year', 'competiter']


In [59]:
print('training data processed')

params = {"objective": "reg:linear",
          "booster" : "gbtree",
          "eta": 0.02,
          "max_depth": 10,
          "subsample": 0.9,
          "colsample_bytree": 0.7,
          "silent": 1,
          "seed": 1301
          }
num_boost_round = 1000

training data processed


In [60]:
'''
# for custom-feature
features=['Store',
 'CompetitionDistance',
 'Promo',
 'Promo2',
 'Promo2SinceWeek',
 'Promo2SinceYear',
 'SchoolHoliday',
 'StoreType',
 'Assortment',
 'StateHoliday',
 'DayOfWeek',
 'month',
 'day',
 'year',
 'competiter',
 'Customers_mean',
 'values_mean']
'''

"\n# for custom-feature\nfeatures=['Store',\n 'CompetitionDistance',\n 'Promo',\n 'Promo2',\n 'Promo2SinceWeek',\n 'Promo2SinceYear',\n 'SchoolHoliday',\n 'StoreType',\n 'Assortment',\n 'StateHoliday',\n 'DayOfWeek',\n 'month',\n 'day',\n 'year',\n 'competiter',\n 'Customers_mean',\n 'values_mean']\n"

In [61]:
print("Train a XGBoost model")
X_train, X_valid = train_test_split(train, test_size=0.012, random_state=10)
y_train = np.log1p(X_train.Sales)
y_valid = np.log1p(X_valid.Sales)
dtrain = xgb.DMatrix(X_train[features], y_train)
dvalid = xgb.DMatrix(X_valid[features], y_valid)

Train a XGBoost model


In [63]:
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
gbm = xgb.train(params, dtrain, num_boost_round, evals=watchlist, \
  early_stopping_rounds=100, feval=rmspe_xg, verbose_eval=True)

Will train until eval error hasn't decreased in 100 rounds.
[0]	train-rmspe:0.999998	eval-rmspe:0.999853
[1]	train-rmspe:0.999996	eval-rmspe:0.999783
[2]	train-rmspe:0.999993	eval-rmspe:0.999709
[3]	train-rmspe:0.999989	eval-rmspe:0.999629
[4]	train-rmspe:0.999983	eval-rmspe:0.999543
[5]	train-rmspe:0.999974	eval-rmspe:0.999447
[6]	train-rmspe:0.999963	eval-rmspe:0.999338
[7]	train-rmspe:0.999948	eval-rmspe:0.999212
[8]	train-rmspe:0.999927	eval-rmspe:0.999068
[9]	train-rmspe:0.999900	eval-rmspe:0.998907
[10]	train-rmspe:0.999866	eval-rmspe:0.998726
[11]	train-rmspe:0.999820	eval-rmspe:0.998524
[12]	train-rmspe:0.999760	eval-rmspe:0.998299
[13]	train-rmspe:0.999683	eval-rmspe:0.998048
[14]	train-rmspe:0.999587	eval-rmspe:0.997769
[15]	train-rmspe:0.999471	eval-rmspe:0.997459
[16]	train-rmspe:0.999324	eval-rmspe:0.997120
[17]	train-rmspe:0.999137	eval-rmspe:0.996745
[18]	train-rmspe:0.998899	eval-rmspe:0.996336
[19]	train-rmspe:0.998610	eval-rmspe:0.995884
[20]	train-rmspe:0.998267	eval

In [64]:
print("Validating")
yhat = gbm.predict(xgb.DMatrix(X_valid[features]))
error = rmspe(X_valid.Sales.values, np.expm1(yhat))
print('RMSPE: {:.6f}'.format(error))

Validating
RMSPE: 0.663088


# submission

In [22]:
dtest = xgb.DMatrix(test[features])
test_probs = gbm.predict(dtest)
# Make Submission
result = pd.DataFrame({"Id": test["Id"], 'Sales': np.expm1(test_probs)})
result.to_csv("xgboost_10_submission.csv", index=False)

# feature importance

In [337]:
create_feature_map(features)
importance = gbm.get_fscore(fmap='xgb.fmap')
importance = sorted(importance.items(), key=operator.itemgetter(1))

df = pd.DataFrame(importance, columns=['feature', 'fscore'])
df['fscore'] = df['fscore'] / df['fscore'].sum()

featp = df.plot(kind='barh', x='feature', y='fscore', legend=False, figsize=(6, 10))
plt.title('XGBoost Feature Importance')
plt.xlabel('relative importance')
fig_featp = featp.get_figure()
fig_featp.savefig('feature_importance_xgb.png', bbox_inches='tight', pad_inches=1)
